In [71]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

In [75]:
df = pd.read_csv('../CSV Files/df_ufc_masters_w_reversed.csv')

In [76]:
features = [
    'B_avg_SIG_STR_landed',
    'B_avg_SIG_STR_pct',
    'B_avg_SUB_ATT',
    'B_avg_TD_landed',
    'B_avg_TD_pct',
    'R_avg_SIG_STR_landed',
    'R_avg_SIG_STR_pct',
    'R_avg_SUB_ATT',
    'R_avg_TD_landed',
    'R_avg_TD_pct',
    'B_Height_cms',
    'B_Reach_cms',
    'R_Height_cms',
    'R_Reach_cms', 
    'R_age',
    'B_age'
]

# List of columns to apply np.log
columns_to_log = ['B_avg_SIG_STR_landed', 'B_avg_SUB_ATT', 'B_avg_TD_landed', 
                  'R_avg_SIG_STR_landed', 'R_avg_SUB_ATT', 'R_avg_TD_landed']

# Applying np.log to each column in the list
for col in columns_to_log:
    # Adding a small constant to avoid log(0) which is undefined
    df[col] = np.log(df[col] + 1e-5)


In [77]:
df['date'] = pd.to_datetime(df['date'])

In [78]:
type(df['date'])

pandas.core.series.Series

In [ ]:
train_end_date = '09-03_2020'
test_start_date = 'your_test_start_date'

In [ ]:
X =df[features]
y= df['Winner']

In [ ]:
y_encoded = y.apply(lambda x: 1 if x == 'Red' else 0)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [ ]:
sScaler = StandardScaler()
X_train_scaled = sScaler.fit_transform(X_train)
X_test_scaled = sScaler.transform(X_test)

In [ ]:
random_forest_model = RandomForestClassifier(n_estimators=50,max_depth=3,min_samples_split=5, random_state=42)
random_forest_model.fit(X_train_scaled, y_train)

RandomForestClassifier(max_depth=3, min_samples_split=5, n_estimators=50,
                       random_state=42)

In [ ]:
y_pred_test = random_forest_model.predict(X_test_scaled)
y_pred_train = random_forest_model.predict(X_train_scaled)

In [ ]:
y_pred_percent= random_forest_model.predict_proba(X_test_scaled)

In [ ]:
y_pred_proba_df = pd.DataFrame(y_pred_percent, columns=['Probability_Blue', 'Probability_Red'])
result_df = pd.concat([df, y_pred_proba_df], axis=1)

In [ ]:
result=result_df[['R_fighter', 'B_fighter', 'Winner', 'R_odds', 'B_odds','R_ev', 'B_ev', 'Probability_Blue', 'Probability_Red']].copy().dropna()

In [ ]:
#B_ev * Prob Blue - Prob Red*100>0
#lage graf for ideele treshold
#hvor mye penger

In [ ]:
condition_blue = (result_df['B_ev'] * result_df['Probability_Blue']-100*result_df['Probability_Red']) > 10
condition_red = (result_df['R_ev'] * result_df['Probability_Red'] - 100*result_df['Probability_Blue']) > 10
condition_blue.sum()
result_df.shape

(9788, 110)

In [ ]:
bank=0
for index, row in result_df.iterrows():

    if condition_blue[index]:
        bet_amount = 100  
        print(f"Who Win: {row['Winner']}, Win price: {row['B_ev']}, Win prob: {row['Probability_Blue']}")
        if row['Winner'] == 'Blue':
            #bank +=1
            bank += row['B_ev'] * (bet_amount / 100)
        else:
            bank -= bet_amount
        print(f'Bank amount: {bank}')
print(bank)

Who Win: Red, Win price: 130.0, Win prob: 0.4848939374915735
Bank amount: -100
Who Win: Red, Win price: 475.0, Win prob: 0.5298314023469617
Bank amount: -200
Who Win: Blue, Win price: 135.0, Win prob: 0.5212490368468566
Bank amount: -65.0
Who Win: Red, Win price: 225.0, Win prob: 0.5053556487782758
Bank amount: -165.0
Who Win: Red, Win price: 135.0, Win prob: 0.4929508782708469
Bank amount: -265.0
Who Win: Red, Win price: 235.0, Win prob: 0.5219479364958274
Bank amount: -365.0
Who Win: Red, Win price: 155.0, Win prob: 0.48531402232165716
Bank amount: -465.0
Who Win: Red, Win price: 800.0, Win prob: 0.44844585792374647
Bank amount: -565.0
Who Win: Red, Win price: 290.0, Win prob: 0.5189157350331377
Bank amount: -665.0
Who Win: Red, Win price: 180.0, Win prob: 0.5229529328936151
Bank amount: -765.0
Who Win: Red, Win price: 385.0, Win prob: 0.49152099088241696
Bank amount: -865.0
Who Win: Red, Win price: 130.0, Win prob: 0.5156828728052648
Bank amount: -965.0
Who Win: Red, Win price: 150.

In [ ]:
bank=0
for index, row in result_df.iterrows():
    if condition_blue[index]:
        bet_amount = 1  
        
        if row['Winner'] == 'Blue':
            bank +=1
            #bank += row['B_ev'] * (bet_amount / 100)
        else:
            bank -= bet_amount
    elif condition_red[index]:
        bet_amount = 1  
        
        if row['Winner'] == 'Red':
            bank +=1
            #bank += row['B_ev'] * (bet_amount / 100)
        else:
            bank -= bet_amount
print(bank)

-559


In [ ]:
accuracy = accuracy_score(y_train, y_pred_train)
conf_matrix = confusion_matrix(y_train, y_pred_train)
classification_rep = classification_report(y_train, y_pred_train)

print(f"Accuracy: {accuracy:.4f}")
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", classification_rep)

Accuracy: 0.5980
Confusion Matrix:
 [[2388 1541]
 [1607 2294]]
Classification Report:
               precision    recall  f1-score   support

           0       0.60      0.61      0.60      3929
           1       0.60      0.59      0.59      3901

    accuracy                           0.60      7830
   macro avg       0.60      0.60      0.60      7830
weighted avg       0.60      0.60      0.60      7830



In [ ]:
accuracy = accuracy_score(y_test, y_pred_test)
conf_matrix = confusion_matrix(y_test, y_pred_test)
classification_rep = classification_report(y_test, y_pred_test)

print(f"Accuracy: {accuracy:.4f}")
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", classification_rep)

Accuracy: 0.5735
Confusion Matrix:
 [[572 393]
 [442 551]]
Classification Report:
               precision    recall  f1-score   support

           0       0.56      0.59      0.58       965
           1       0.58      0.55      0.57       993

    accuracy                           0.57      1958
   macro avg       0.57      0.57      0.57      1958
weighted avg       0.57      0.57      0.57      1958

